# Лекция 10: Параллельные вычисления

__Автор: Сергей Вячеславович Макрушин__ e-mail: SVMakrushin@fa.ru 

Финансовый универсиет, 2021 г. 

При подготовке лекции использованы материалы:
* ...

V 0.4 07.11.2021

## Разделы: <a class="anchor" id="разделы"></a>
* [Введение в параллельные вычисления](#введение)
* [Принципы разработки параллельных алгоритмов](#принципы)
    * [Модели параллельного прграммирования](#модели-пп)
    * [Введение](#датафрэйм-введение)
* [Процессы и потоки](#процессы-и-потоки)    
* [Параллельных вычисления на Python, модуль multiprocessing](#пв-на-python)    

-

* [к оглавлению](#разделы)

In [16]:
# загружаем стиль для оформления презентации
from IPython.display import HTML
from urllib.request import urlopen
html = urlopen("file:./lec_v2.css")
HTML(html.read().decode('utf-8'))

# Введение в параллельные вычисления <a class="anchor" id="введение"></a>
-
* [к оглавлению](#разделы)

__Параллельные вычисления__ (parallel computing)

Традиционно ПО пишется для __последовательных вычислений__:
* задача разбивается на последовательность инструкций
* инструкции выполняются последовательно
* только одна инструкция может исполнятся в один момент времени
* выполняется на одном ЦП (физически или, как минимум, логически)

<center>         
    <img src="./img/pp1.png" alt="Последовательные вычисления" style="width: 500px;"/>
    <b>Последовательные вычисления</b>
</center>

Параллельные вычисления (parallel computing) позволяют решать задачу одновременно используя несколько вычислительных ресурсов:
* выполняется сразу на нескольких ЦП (ядрах)
* задача разбивается на части (tasks), которые могут решаться одновременно
* каждая часть разбивается на последовательность инструкций
* инструкции из разных частей могут исполнятся одновременно на разных ЦП

<center>         
    <img src="./img/pp2.png" alt="Параллельные вычисления" style="width: 500px;"/>
    <b>Параллельные вычисления</b>
</center>

__Пример распараллеливания вычислений__

* Задача: $\sum a_i, i=1,..,n$
* Последовательные вычисления:
    * выполняются на одном ЦП
    * n-1 шаг

* Параллельные вычисления:
    * выполняются на n/2 ЦП
    * $\log_2(n)$ шагов

<center>         
    <img src="./img/pp3.png" alt="Параллельные вычисления" style="width: 700px;"/>
    <b>Пример распараллеливания суммирования</b>
</center>

__Причины использования параллельных вычислений__

Параллельные вычисления используются для:
* решения больших задач (время решения с помощью одного исполнителя неприемлемо)
* ускорения решения задачи
* обеспечения одновременного решения нескольких задач
* эффективного использования доступных вычислительных ресурсов (нескольких ядер на ЦП, нескольких компьютеров и т.д.)

__Но__, нужно понимать, что:
* <em class="mn"></em> в большинстве случаев __создание параллельной реализации задачи более трудоемко__ и должно быть оправдано преимуществами от использования такой реализации
* <em class="mn"></em> __эффективность использования вычислительных ресурсов__ при параллельных вычислениях __ниже__, чем при последовательных вычислениях из-за:
    * накладных расходов обмена данных
    * накладных расходов синхронизации и вынужденных простоев

__Архитектура фон Неймана__

* __Принцип двоичного кодирования__: вся информация, как данные (в т.ч. адреса), так и команды, кодируются двоичными цифрами 0 и 1.
* __Принцип адресности__: основная память состоит из пронумерованных ячеек, процессору в произвольный момент доступна любая ячейка, для доступа к ячейкам используются их номера – адреса.
* __Принцип программного управления__: программа состоит из команд, предписывающих операцию из заданного набора операций. Команды программы хранятся в последовательных ячейках памяти вычислительной машины и выполняются последовательно в заданной очередности.
* __Принцип однородности памяти__: команды и данные хранятся в одной и той же памяти и внешне в памяти неразличимы. Распознать их можно только по способу использования.

<center>         
    <img src="./img/pp4.png" alt="Архитектура фон Нейман" style="width: 550px;"/>
    <b>Архитектура фон Неймана</b>
</center>

__Ресурсы для параллельных вычислений__

Вычислительные ресурсы для параллельных вычислений могут представлять из себя:
* один компьютер с несколькими ЦП (и/или ядрами)
* один компьютер с ЦП (несколькими ЦП) и специализированными вычислительными ресурсами, например^
    * графическим процессором (процессорами)  (graphics processing unit, GPU)
    * application-specific integrated circuit (ASIC) - интегральной схемой (микросхемой) разработанной для специализированного использования
    * Tensor Processing Unit (TPU) - тензорный процессор, относящийся к классу нейронных процессоров, являющийся специализированной интегральной схемой (ASIC), разработанной корпорацией Google
* несколько компьютеров, объединенных в сеть
* комбинацию предыдущих вариантов

__Классификация Флинна__

<center>         
    <img src="./img/pp5.png" alt="Классификация Флинна" style="width: 650px;"/>
    <b>Классификация Флинна</b>
</center>

__Классификация MIMD-систем__


<center>         
    <img src="./img/mimd.png" alt="Классификация MIMD-систем" style="width: 650px;"/>
    <b>Классификация MIMD-систем</b>
</center>

__Архитектура с общей памятью__

Принципы многопроцессорной архитектуры с общей памятью (shared memory):
* несколько процессоров работают независимо, но совместно используют общую память
* изменения в памяти осуществляемые одним процессором видны всем другим процессорам

<center>         
    <img src="./img/uma-numa.png" alt="Архитектуры UMA и NUMA" style="width: 650px;"/>
    <b>Архитектуры UMA и NUMA</b>
</center>

Типы реализации архитектуры с общей памятью:
* __однородный доступ к памяти (Uniform Memory Access, UMA)__ – равный (в т.ч. по времени) доступ всех процессоров ко всем областям основной памяти (процессоры могут иметь свой (когерентный) кэш). Типично для архитектуры с симметричной мультипроцессорностью (Symmetric Multiprocessing,  SMP).
* __неоднородный доступ к памяти (Non-Uniform Memory Access, NUMA)__ – время доступа к памяти определяется её расположением по отношению к процессору. Обычно реализуется как соединение нескольких SMP узлов.

Преимущества и недостатки:
* <em class="pl"></em> Привычная модель программирования за счет единого адресного пространства
* <em class="pl"></em> Высокая скорость и низкая латентность обмена данными между параллельными задачами
* <em class="mn"></em> Низкая масштабируемость (обычно до 16 процессоров) из-за геометрического роста нагрузки на шину CPU-RAM
* <em class="mn"></em> Проблема поддержания когерентности кэшей
* <em class="mn"></em> Трудоемкая организация эффективного использование памяти в NUMA-системах
* <em class="mn"></em> Необходимость синхронизации при доступе к общим данным (критические секции)

__Архитектура с распределенной памятью и гибридная архитектура__

Принципы многопроцессорной архитектуры с распределенной памятью (distributed memory):
* несколько процессоров работают с собственной памятью, недоступной напрямую для других процессоров (отсутствует общая адресация памяти)
* обмен данными между процессорами производится через коммуникационную сеть и явно определяется исполняемой программой

<center>         
    <img src="./img/dma.png" alt="Архитектура с распределенной памятью" style="width: 450px;"/>
    <b>Архитектура с распределенной памятью</b>
</center>

Реализации архитектуры с распределенной памятью:
* возможно большое количество вариантов организации коммуникационной сети между узлами архитектуры с распределенной памятью
* на практике часто узлами систем с распределенной памятью являются многопроцессорные узлы с общей памятью (гибридная архитектура)

Преимущества и недостатки:
* <em class="pl"></em> Высокая масштабируемость
* <em class="pl"></em> Объем памяти растет пропорционально количеству ядер
* <em class="pl"></em> Возможность использовать недорогие массовые компоненты
* <em class="mn"></em> Специальные подходы к программированию: необходимость использования передачи сообщений (message passing)
* <em class="mn"></em> Сложность реализации некоторых структур данных и алгоритмов
* <em class="mn"></em> Высокая латентность и низкая скорость обмена данными между узлами
* <em class="mn"></em> Неоднородность, отказы узлов


<center>         
    <img src="./img/ga.png" alt="Гибридная архитектура" style="width: 650px;"/>
    <b>Гибридная архитектура</b>
</center>

# Принципы разработки параллельных алгоритмов <a class="anchor" id="принципы"></a>
-
* [к оглавлению](#разделы)

__Разработка параллельного алгоритма__

Ключевые шаги разработки параллельного алгоритма:
1. __Поиск параллелизма__ в известном последовательном алгоритме, его модификация или создание нового алгоритма
2. __Декомпозиция__ задачи на подзадачи, которые могут выполняться параллельно
3. __Анализ зависимостей__ между подзадачами

<b class="r">NB!</b> Параллельная версия самого эффективного последовательного алгоритма решения задачи необязательно будет самой эффективной параллельной реализацией для рассматриваемой задачи.

Специфические задачи реализации параллельного алгоритма в виде параллельной программы:
* Распределение подзадач между процессорами (task mapping, load balancing)
* Организация взаимодействия подзадач (message passing, shared data structures)

Создание эффективной параллельной реализации алгоритма требует:
* Учета архитектуры целевой вычислительной системы
* Измерения и анализа показателей эффективности параллельной  программы 
* Оптимизации программы

__Показатели эффективности параллельных алгоритмов__

__Закон Амдала__ - иллюстрирует ограничение роста производительности решения задачи при ее распараллеливании.

Обозначения:
* доля задачи $1-\alpha$ может быть распараллелена идеально 
* доля задачи $\alpha$ от общего объёма вычислений может быть получена только последовательными расчётами (выполняется только в один поток)
* $p$ - число задействованных узлов 

Ускорение, которое может быть получено на вычислительной системе из $p$ процессоров, по сравнению с однопроцессорным решением не будет превышать величины:


$$S_p = \cfrac{1}{\alpha + \cfrac{1 - \alpha}{p}}$$

<center>         
    <img src="./img/amdal_2.png" alt="Закон Амдала" style="width: 350px;"/>
    <b>Закон Амдала</b>
</center>


__Задачи параллельного программирования__

Параллельное программирование требует :
* Выделения __подзадач__, которые могут выполняться параллельно
* Определения __данных__, которые должны разделятся/пересылаться __между подзадачами__
* __Синхронизации__ подзадач

В зависимости от задач можно проводить крупнозернистую (coarse-grained granularity) и мелкозернистую (fine-grained granularity) декомпозицию:
* __крупнозернистая__ – задача разбивается на небольшое количество крупных блоков
    * <em class="pl"></em>: снижается обмен информацией и затраты на синхронизацию
    * <em class="mn"></em>:  несбалансированность нагрузки, 
    * <em class="mn"></em>: ограничение по степени параллелизма
* __мелкозернистая__ – задача разбивается на большое количество небольших блоков
    * часто выявление мелкозернистой декомпозиции выполняется компилятором

__Подходы к декомпозции на подзадачи__

Два основных подхода к декомпозиции задач на параллелизуемые подзадачи:

__Функциональная декомпозиция__ (Task/Functional decomposition) 
* Распределение вычислений по подзадачам

<center>         
    <img src="./img/decompose1.png" alt="Функциональная декомпозиция" style="width: 450px;"/>
    <b>Функциональная декомпозиция</b>
</center>

__Декомпозиция по данным__ (Domain/Data decomposition)
* Распределение данных по подзадачам
* Высокая масштабируемость (многие тысячи ядер)
* Возможность использовать недорогие массовые компоненты (CPU, RAM, сети)

<center>         
    <img src="./img/decompose2.png" alt="Декомпозиция по данным" style="width: 450px;"/>
    <b>Декомпозиция по данным</b>
</center>


<center>         
    <img src="./img/decompose3.png" alt="Подходы к декомпозиции задач" style="width: 650px;"/>
    <b>Подходы к декомпозиции задач</b>
</center>

__Конвейерная обработка данных__
_by flow data -(regular)-> Pipeline_
* Имеется регулярных поток блоков данных, каждый из которых проходит несколько стадий обработки – выполняемых на этапах конвейера

<center>         
    <img src="./img/decompose4.png" alt="Пример конвейерной обработки данных" style="width: 550px;"/>
    <b>Пример конвейерной обработки данных</b>
</center>

__Геометрическая декомпозиция__
_by data -(linear)-> Geometric decomposition_
* Данные задачи разбиваются на области (желательно равного размера) по "геометрическому" принципу (например n-мерная решетка с регулярным шагом)
* С каждой областью данных ассоциирется свой обработчик, обычно применяющий стандартный алгоорим обработки и при необходимости обменивающийся данными с обработчиками, работающими с соседними областями.

<center>         
    <img src="./img/decompose5.png" alt="Пример конвейерной обработки данных" style="width: 350px;"/>
    <b>Пример геометрической декомпозиции для трехмерной области</b>
</center>

__Рекурсивный параллелизм (разделяй и властвуй)__
_by tasks -(recursive)-> Divide and Conquer_
Операции Split и Merge могут стать узким местом т.к. выполняются последовательно
Задания порождаются динамически (балансировка загрузки потоков)
Степень параллелизма изменяется в ходе выполнения алгоритма

<center>         
    <img src="./img/decompose6.png" alt="Пример конвейерной обработки данных" style="width: 450px;"/>
    <b>Пример рекурсивного параллелизам по принципу "разделяй и властвуй"</b>
</center>

## __Модели параллельного прграммирования__ <a class="anchor" id="модели-пп"></a>
-
* [к оглавлению](#разделы)

__Разделяемая память__ (shared memory): 
* Аналогия  - __доска объявлений__
* Подзадачи используют общее адресное пространство (оперативной памяти)
* Подзадачи __взаимодействуют асинхронно__ читая и записывая информацию в общем пространстве
* Реализация: многопоточные приложения, OpenMP

__Передача сообщений__ (message passing): 
* Аналогия – __отправка писем__ с явным указанием отправителя и получателя
* Каждая подзадача работает с собственными локальными данными
* Подзадачи взаимодействуют за счет обмена сообщениями
* Реализация: MPI (message passing interface)

__Параллельная обработка данных__ (data parallelization):
* Строго описанные глобальные операции над данными
* (Может обозначаеться как чрезвычайная параллельность (embarrassingly parallel) – очень хорошо распараллеливаемые вычисления)
* Обычно данные равномерно разделяются по подзадачам
* Подзадачи выполняются как набор независимых операций
* Реализация может быть сделана как с помощью разделяемой памяти, так и с помощью передачи сообщений

*__Модель параллельного программирования на основе передачи сообщений__*

Основные характеристики модели на основе передачи сообщений:
* Набор задач, имеющих свою собственную локальную память во время вычислений
* Задачи могут находится как на одной машине (в т.ч. с разделяемой памятью), так и на разных машинах
* Задачи обмениваются данными с помощью отсылки и приема сообщений явно описываемых в программном коде
* Зачастую передача данных подразумевает их сериализацию/десериализацию, что требует соответствующих накладных расходов
* Как правило передача данных требует совместной работы, выполняемой как задачей-отправителем, так и задачей-получателем

Программирование для модели на основе передачи сообщений:
* С точки зрения программирования модель на основе передачи сообщений выглядит как внедрение вызовов специализированной библиотеки в программный код.
* За реализацию параллелизма полностью отвечает программист, а не компилятор
* Общепринятым стандартом для модели параллельного программирования на основе передачи сообщений является библиотека MPI (Message Passing Interface).

<center>         
    <img src="./img/pr_thr5.png" alt="Пример передачи сообщения" style="width: 650px;"/>
    <b>Пример передачи сообщения</b>
</center>

* При отправке определяется процесс отправитель и буфер содержащий передаваемую информацию
* При получении определяется процесс получатель и место для сохранения результата
* Возможны дополнительные атрибуты обуславливающие передачу сообщения
* Передача сообщения несет много накладных расходов

-
* Возможны реализации не двухточечной коммуникации с дополнительными структурами данных, например, очередями сообщений

*__Модель параллельного программирования на основе параллельной обработки данных__*

Параллельная обработка данных (data parallelization):
* Строго описанные глобальные операции над данными (может обозначаться как чрезвычайная параллельность (embarrassingly parallel) – очень хорошо распараллеливаемые вычисления)
* Обычно данные равномерно разделяются по подзадачам
* Подзадачи выполняются как последовательность независимых операций
* Реализация может быть сделана как с помощью разделяемой памяти, так и с помощью передачи сообщений

Основные характеристики модели на основе параллельной обработки данных:
* Основные параллельные задачи сфокусированы на выполнении операций над неким массивом данных
* __Массив данных__ обычно организован в виде __однородной структуры__, например массива или гиперкуба
* Задачи обычно параллельно выполняют аналогичные операции над выделенными им фрагментами одного массива данных
* В реализации на архитектурах без разделяемой памяти массив данных делится на фрагменты, которые находятся в распоряжении отдельных задач
* Программирование для данной модели обычно представляет собой написания программы оперирующей с конструкциями для параллельной обработки данных, например в виде вызовов специализированной библиотеки

# Процессы и потоки <a class="anchor" id="процессы-и-потоки"></a>

-

* [к оглавлению](#разделы)

<table border="1" id="cssTableCenter" class="docutils">
    <colgroup>
        <col width="30%">
        <col width="30%">
    </colgroup>
    <thead valign="bottom">
        <tr class="row-odd">
            <th class="head" style="text-align: center; vertical-align: middle;">Процесс (process)</th>
            <th class="head" style="text-align: center; vertical-align: middle;">Поток (thread)</th>
        </tr>
    </thead>
    <tbody valign="top">
        <tr class="row-even">
            <td>Процесс – полноценная программа (использует большое количество системных ресурсов)</td>
            <td>Поток – часть процесса (создается существенно проще)</td>
        </tr>
        <tr class="row-odd">
            <td>Разные процессы имеют изолированное адресное пространство</td>
            <td>Потоки одного процесса разделяют общую память (и другие ресурсы)</td>
        </tr>
        <tr class="row-even">
            <td>Процессы взаимодействуют через системные механизмы межпроцессной коммуникации</td>
            <td>Потоки взаимодействуют через разделяемую память</td>
        </tr>
        <tr class="row-odd">
            <td>
                <img src="./img/pr_thr1.png" alt="Пример конвейерной обработки данных" style="width: 150px;"/>               
            </td>                
            <td>
                <img src="./img/pr_thr2.png" alt="Пример конвейерной обработки данных" style="width: 250px;"/>               
            </td>
        </tr>  
    </tbody>
</table>

__Мнгопоточный приложения__

Специфика работы многопоточного приложения: 
* Процесс начинает исполнятся в своей входной точке как поток
* Потоки могут создавать другие потоки внутри процесса
* Все потоки внутри процесса __разделяют блоки с кодом и общими данными__ 

Сценарий работы многопоточного приложения: 
* Сначала инициализируется стартовый поток процесса, выполняет загрузку необходимых ресурсов
* Стартовый поток выполняет последовательные задачи и затем создает новые потоки, выполняемые внутри общего процесса, для параллельного выполнения подзадач
* Каждый поток имеет собственные данные, но разделяет общие ресурсы процесса
* Потоки взаимодействуют через глобальную память процесса (корректное изменение общих данных требует использования механизмов синхронизации)
* Параллельная работа потоков может быть представлена как параллельное исполнение процедур внутри общей программы.

<center>         
    <img src="./img/pr_thr3.png" alt="Схема совместной работы потоков" style="width: 450px;"/>
    <b>Схема совместной работы потоков</b>
</center>

__Подзадачи и потоки__

* __Задача__ (подзадача) состоит из данных и процедуры их обработки
* __Планировщик задач__ назначает задачу для исполнения в одном из потоков

Приемущества подхода:
* <em class="pl"></em> Оперирование задачами намного более простое, чем потоками
* <em class="pl"></em> Работа планировщика позволяет балансировать нагрузку между потоками

Но:
* Задач должно быть намного больше, чем потоков: это обеспечивает гибкость назначения задач и простоту балансировки
* Объем вычислений в задаче должен быть достаточно большим, чтобы накладные расходы по управлению задачами были оправданы

<center>         
    <img src="./img/pr_thr4.png" alt="Организаця работы с задачами" style="width: 650px;"/>
    <b>Организаця работы с задачами</b>
</center>

__Проблемы синхронизации__

__Взаимная блокировка (deadlock)__ – ситуация в многозадачной среде при которой несколько потоков находятся в состоянии ожидания ресурсов, занятых друг другом, и ни один из них не может продолжать свое выполнение
* Типичная взаимная блокировка: два потока ожидают окончания друг друга

__Состояние гонки (race condition)__ – ситуация в которой работа приложения зависит от того, в каком порядке выполнятся (параллельные) части кода
* Несколько потоков модифицируют разделяемый ресурс (например, переменную)
* Результат зависит от того, какой поток первым выполнит изменения
* Проблема может быть решена за счет блокировок, но зачастую это сложно, и приводит к ошибкам, в частности, взаимной блокировке

__Потоковая безопасность (thread safety)__ –  специфика кода (например функций или библиотек), позволяющая использовать его из нескольких потоков одновременно
* Источником нарушения потоковой безопасности может быть: 
    * доступ к глобальным переменным или динамической памяти
    * выделение/освобождение глобальных ресурсов (например файлов)
    * неявный доступ через указатели
    * побочный эффект функций
* Эффективным подходом является изменение только локальных переменных потока.

# Параллельных вычисления на Python, модуль multiprocessing <a class="anchor" id="пв-на-python"></a>

-

* [к оглавлению](#разделы)

__Проблема Global Interpreter Lock (GIL)__

__Global Interpreter Lock__ – __способ синхронизации потоков__ используемый в рефернсной реализации Python (CPython) и в реализациях некоторых других интерпретируемых языков программирования. 

* Интерпретатор CPython __НЕ является потоково-безопасным__ т.к. некоторые ключевые структуры данных могут быть одновременно доступны только одному потку. 
* GIL является самым __простым и быстрым при исполнении однопоточных приложений__ способом обеспечения потоковой безопасности при одновременном обращении разных потоков к одним и тем же участкам памяти.
* Наличие __GIL не является требованием языка__ программирования Python, а только спецификой реализации самого популярного интерпретатора CPython, существуют другие интерпретаторы Python не имеющие GIL.

Для __обхода проблемы GIL__ для реализации параллельных вычислений в Python вместо многопоточного подхода с разделяемой памятью используется более тяжеловесная конструкция: 
* множество процессов, в каждом из которых работает собственный интерпретатор с собственным GIL и имеется собственная копия данных и кода
    * обмен данными между процессами обычно производится не через разделяемую память (это иногда возможно, но чревато ошибками), а через передачу данных и кода с помощью сериализации
    * по сути это вариация на тему модели параллельного программирования на основе передачи сообщений (реализуемой в т.ч. при вычислении на компьютере с разделяемой памятью)

__Библиотеки Python для параллельных вычислений__

Библиотеки Python для параллельных вычислений:
* … тысячи их: http://wiki.python.org/moin/Parallel

В стандартной библиотеке Python:
* __threading__ – обеспечение параллельных вычислений на основе потоков и поддержка работы с блокировками
* __multiprocessing__ – обеспечение параллельных вычислений на основе процессов и поддержка соответствующей инфраструктуры



<table border="1" id="cssTableCenter" class="docutils">
    <colgroup>
        <col width="30%">
        <col width="30%">
    </colgroup>
    <thead valign="bottom">
        <tr class="row-odd">
            <th class="head" style="text-align: center; vertical-align: middle;">threading</th>
            <th class="head" style="text-align: center; vertical-align: middle;">multiprocessing</th>
        </tr>
    </thead>
    <tbody valign="top">
        <tr class="row-even">
            <td>
                <ul>
                    <li><em class="pl"></em> Легковесный, позволяет использовать разделяемую память</li>
                    <li><em class="pl"></em> Подходит для параллельных приложений использующих расширения на C корректно освобождающие GIL или связанные с активным использованием ввода/вывода</li>
                </ul>               
            </td>
            <td>             
                <ul>
                    <li><em class="pl"></em> Позволяет избегать ограничения GIL для параллельных вычислений</li>
                    <li><em class="pl"></em> Позволяет избегать использования примитивов для синхронизации (модель передачи сообщений)</li>
                    <li><em class="pl"></em> Включает абстракции с интерфейсом, похожим на threading.Thread</li>
                </ul>                               
            </td>
        </tr>
        <tr class="row-odd">
            <td>
                <ul>
                    <li><em class="mn"></em> Подвержен влиянию GIL</li>
                    <li><em class="mn"></em> Требует исопльзования примитивов для синхронизации (кроме модели очереди команд)</li>
                </ul>               
            </td>
            <td>             
                <ul>
                    <li><em class="mn">Более сложная и несет больше накладных расходов</em> </li>
                </ul>                               
            </td>
        </tr>       

    </tbody>
</table>

## Introduction to the `multiprocessing` module

Модуль [multiprocessing](https://docs.python.org/dev/library/multiprocessing.html) включен в стандартную библиотеку Python. Здесь можно познакомиться с официальной документацией к модулю: [official documentation](https://docs.python.org/dev/library/multiprocessing.html) .  



__Класс `Process`__

Самый простой подход заключается в использовании класса Process из модуля multiprocessing. Рассмотрим пример с использованием простой функции очереди для параллельной генерации четырех случайных строк.

In [17]:
%%file rand_string_.py

import random
import string

def rand_string(length, output):
    """ Generates a random string of numbers, lower- and uppercase chars. """
    rand_str = ''.join(random.choice(
                        string.ascii_lowercase 
                        + string.ascii_uppercase 
                        + string.digits)
                   for i in range(length))
    output.put(rand_str)

Overwriting rand_string_.py


In [18]:
import rand_string_

In [19]:
import multiprocessing as mp
import random
import string

random.seed(123)

# Define an output queue
output = mp.Queue()

# Setup a list of processes that we want to run
processes = [mp.Process(target=rand_string_.rand_string, args=(5, output)) \
             for x in range(4)]

# Run processes
for p in processes:
    p.start()

# Exit (wait exit) the completed processes
for p in processes:
    p.join()

# Get process results from the output queue
results = [output.get() for p in processes]

print(results)

['s8Kf8', '3qpdC', 'vtr6m', 'PIP29']


### How to retrieve results in a particular order 

__Как получить результат в конкретном порядке?__

Порядок полученных результатов не обязательно будет совпадать с порядком процессов (в списке `processes`). Поскольку мы в конечном итоге используем метод `.get()` для последовательного получения результатов из `Queue`, порядок, в котором завершаются процессы, определяет порядок наших результатов.
Например, если второй процесс завершится до первого процесса, порядок строк в списке `results` может быть иным:
`['PQpqM', 'yzQfA', 'SHZYV', 'PSNkD']` вместо `['yzQfA', 'PQpqM', 'SHZYV', 'PSNkD']`

Если наше приложение требует, чтобы мы извлекали результаты в определенном порядке, можно использовать атрибут процесса `._identity`. В этом случае мы также могли бы просто использовать значения из нашего объекта `range` в качестве аргумента для хранения номера процесса. Измененный код будет:

In [20]:
%%file rand_string_2.py

import random
import string

def rand_string(length, pos, output):
    """ Generates a random string of numbers, lower- and uppercase chars. """
    rand_str = ''.join(random.choice(
                        string.ascii_lowercase 
                        + string.ascii_uppercase 
                        + string.digits)
                   for i in range(length))
    output.put((pos, rand_str))

Overwriting rand_string_2.py


In [21]:
import rand_string_2

In [22]:
# Define an output queue
output = mp.Queue()

# define a example function
# def rand_string(length, pos, output):
#     """ Generates a random string of numbers, lower- and uppercase chars. """
#     rand_str = ''.join(random.choice(
#                         string.ascii_lowercase 
#                         + string.ascii_uppercase 
#                         + string.digits)
#                    for i in range(length))
#     output.put((pos, rand_str))

# Setup a list of processes that we want to run
processes = [mp.Process(target=rand_string_2.rand_string, args=(5, x, output)) for x in range(4)]

# Run processes
for p in processes:
    p.start()

# Exit the completed processes
for p in processes:
    p.join()

# Get process results from the output queue
results = [output.get() for p in processes]

print(results)

[(0, 'sAFAz'), (1, 'ogciO'), (2, 'IP7gl'), (3, 'JwgLm')]


Результатами работы функций будут кортежи, например: `[(0, 'KAQo6'), (1, '5lUya'), (2, 'nj6Q0'), (3, 'QQvLr')]`   
or `[(1, '5lUya'), (3, 'QQvLr'), (0, 'KAQo6'), (2, 'nj6Q0')]` .

Для того, чтобы убедиться что мы получили результаты в нужном порядке мы можем просто отсортировать их (и при необходимости избавиться от информации о номере процесса, попродившего результат):

In [23]:
results.sort()
results = [r[1] for r in results]
print(results)

['sAFAz', 'ogciO', 'IP7gl', 'JwgLm']


---

Пример, показывающий, как использовать очереди для подачи задач в пул рабочих процессов и собирать результат их работы:

In [8]:
%%file my_worker.py

# import random
# import string
import time
import random

from multiprocessing import Process, Queue, current_process, freeze_support

def worker(input_, output):
    for func, args in iter(input_.get, 'STOP'):
        result = calculate(func, args)
        output.put(result)
        
#
# Function used to calculate result
#

def calculate(func, args):
    result = func(*args)
    return '%s says that %s%s = %s' % \
        (current_process().name, func.__name__, args, result)        
        
#
# Functions referenced by tasks
#

def mul(a, b):
    time.sleep(0.5*random.random())
    return a * b

def plus(a, b):
    time.sleep(0.5*random.random())
    return a + b        

Overwriting my_worker.py


In [9]:
import my_worker

In [10]:
import time
import random

from multiprocessing import Process, Queue, current_process, freeze_support

# #
# # Function run by worker processes
# #

# def worker(input_, output):
#     for func, args in iter(input_.get, 'STOP'):
#         result = calculate(func, args)
#         output.put(result)

# #
# # Function used to calculate result
# #

# def calculate(func, args):
#     result = func(*args)
#     return '%s says that %s%s = %s' % \
#         (current_process().name, func.__name__, args, result)

# #
# # Functions referenced by tasks
# #

# def mul(a, b):
#     time.sleep(0.5*random.random())
#     return a * b

# def plus(a, b):
#     time.sleep(0.5*random.random())
#     return a + b

#
#
#

def test():
    NUMBER_OF_PROCESSES = 4
    TASKS1 = [(my_worker.mul, (i, 7)) for i in range(20)]
    TASKS2 = [(my_worker.plus, (i, 8)) for i in range(10)]

    # Create queues
    task_queue = Queue()
    done_queue = Queue()

    # Submit tasks
    for task in TASKS1:
        task_queue.put(task)

    # Start worker processes
    for i in range(NUMBER_OF_PROCESSES):
        Process(target=my_worker.worker, args=(task_queue, done_queue)).start()

    # Get and print results
    print('Unordered results:')
    for i in range(len(TASKS1)):
        print('\t', done_queue.get())

    # Add more tasks using `put()`
    for task in TASKS2:
        task_queue.put(task)

    # Get and print some more results
    for i in range(len(TASKS2)):
        print('\t', done_queue.get())

    # Tell child processes to stop
    for i in range(NUMBER_OF_PROCESSES):
        task_queue.put('STOP')

freeze_support()
test()        
# if __name__ == '__main__':
#     freeze_support()
#     test()

Unordered results:
	 Process-9 says that mul(1, 7) = 7
	 Process-9 says that mul(3, 7) = 21
	 Process-10 says that mul(0, 7) = 0
	 Process-9 says that mul(5, 7) = 35
	 Process-9 says that mul(7, 7) = 49
	 Process-12 says that mul(4, 7) = 28
	 Process-11 says that mul(2, 7) = 14
	 Process-12 says that mul(9, 7) = 63
	 Process-9 says that mul(8, 7) = 56
	 Process-10 says that mul(6, 7) = 42
	 Process-11 says that mul(10, 7) = 70
	 Process-12 says that mul(11, 7) = 77
	 Process-9 says that mul(12, 7) = 84
	 Process-12 says that mul(15, 7) = 105
	 Process-10 says that mul(13, 7) = 91
	 Process-11 says that mul(14, 7) = 98
	 Process-11 says that mul(19, 7) = 133
	 Process-9 says that mul(16, 7) = 112
	 Process-10 says that mul(18, 7) = 126
	 Process-12 says that mul(17, 7) = 119
	 Process-12 says that plus(3, 8) = 11
	 Process-11 says that plus(0, 8) = 8
	 Process-9 says that plus(1, 8) = 9
	 Process-12 says that plus(4, 8) = 12
	 Process-9 says that plus(6, 8) = 14
	 Process-11 says that p

### Класс `Pool`

Более простой способ поддерживать упорядоченный список результатов - использовать функции `Pool.apply` и `Pool.map`, которые мы обсудим в следующем разделе.

Другой, более удобный подход для простых задач параллельной обработки - это класс `Pool`. Особенно интересны четыре метода:
* `Pool.apply`
* `Pool.map`
* `Pool.apply_async`
* `Pool.map_async`

Методы `Pool.apply` и `Pool.map` эквивалентны встроенным методам `apply` и `map`.

Прежде чем мы перейдем к асинхронным вариантам методов `Pool`, давайте рассмотрим простой пример с использованием `Pool.apply` и `Pool.map`. Здесь мы установим количество процессов на 4, что означает, что класс Pool разрешит запускать только 4 процесса одновременно.

О разнице между `apply()`, `apply_async()` и `map()`:
* https://stackoverflow.com/questions/8533318/multiprocessing-pool-when-to-use-apply-apply-async-or-map
* apply(), apply_async() возвращают реузльтат в произвольном порядке
* map() в порядке следования данных в исходной коллекции

In [24]:
%%file cube_.py

def cube(x):
    return x**3

Overwriting cube_.py


In [25]:
import cube_

In [26]:
pool = mp.Pool(processes=4)
results = [pool.apply(cube_.cube, args=(x,)) for x in range(1,7)]
print(results)

[1, 8, 27, 64, 125, 216]


In [27]:
pool = mp.Pool(processes=4)
results = pool.map(cube_.cube, range(1,7))
print(type(results[0]))
print(results)

<class 'int'>
[1, 8, 27, 64, 125, 216]


`Pool.map` и `Pool.apply` блокируют основную программу до тех пор, пока все процессы не будут завершены, что очень полезно, если мы хотим получить результаты в определенном порядке для определенных приложений (потребителей).

Напротив, варианты `async` стартуют все процессы сразу и получат результаты, как только они будут готовы. Еще одно отличие состоит в том, что нам нужно использовать метод `get` после вызова `apply_async()`, чтобы получить возвращаемые значения завершенных процессов. 

In [28]:
pool = mp.Pool(processes=4)
results = [pool.apply_async(cube_.cube, args=(x,)) for x in range(1,7)]
print(type(results[0]))
output = [p.get() for p in results]
print(output)

<class 'multiprocessing.pool.ApplyResult'>
[1, 8, 27, 64, 125, 216]
